In [16]:
# Deze cell importeert een aantal functies en modules die je nodig hebt om deze opdracht te maken.

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

# Start notebook mode
init_notebook_mode(connected=True)

import pandas as pd # we gebruiken pandas om de CSV data te laden en te visualiseren


times = pd.read_csv("timesData.csv")
shanghai = pd.read_csv("shanghaiData.csv")
cwur = pd.read_csv("cwurData.csv")

rankings = {} # rankings[universiteit] = [times, shanghai, cwur]

# institution (cwur)

def rename(university_name):
    univ = university_name.split(',')[0].strip()
    univ = univ.split('(')[0].strip()
    
#     if univ != university_name:
#         print(univ, university_name, sep='\t\t')
        
    return univ

print('times')
for world_rank, year, university_name in zip(times['world_rank'], times['year'], times['university_name']):
    if year not in rankings:
        rankings[year] = {}
    rankings[year][rename(university_name)] = world_rank
    
print('shanghai')
for world_rank, year, university_name in zip(shanghai['world_rank'], shanghai['year'], shanghai['university_name']):
    if year not in rankings :
        continue
        
    if univ
    rankings[year][] += world_rank
    
print('cwur')
for world_rank, year, university_name in zip(cwur['world_rank'], cwur['year'], cwur['institution']):
    if year not in rankings:
        continue
    rankings[year][rename(university_name)] += world_rank
    
for year in rankings:
    print('year')
    for univ in rankings[year]:
        print(univ + ' ' + rankings[year][univ])

# print(times['world_rank'], shanghai['world_rank'], cwur['world_rank'])

# mashed = pd.DataFrame()


times
shanghai
2011
2011
2011
2011
2011
2011
2011
2011
2011
2011
2011
2011
2011
2011
2011
2011
2011
2011


KeyError: 'The Johns Hopkins University'